# Master Notebook

This notebook is supposedly the master homepage (for the analysis section at least) and should direct you to where you need to go.

As it currently stands, different tasks are done and explained in different notebooks, but in the future they may be unified here.

As part of getting that all consolidated here, we need to be able to use the modules that exist. To do this we want to be able to access them from wherever we downloaded them. When I did this, I created a folder called QAOP in the folder I'd been working out of. 

Wherever you put the folder containing the scripts/modules we'll be using that you downloaded, copy it's path and run the cell below.

In [1]:
import sys
#My folder is at: /Users/aidanmcclung/Desktop/Summer_Exoplanets
sys.path.append('/Users/aidanmcclung/Desktop/Summer_Exoplanets') #this lets it know to look here for import statements

As a further note: This notebook is the one that will be uploaded to github. There are a few folders which it would/will need that you will need to create:

- `input`
- `output`
- `myWorking`
- `photometry`

If you don't have these folders, there may be troubles. You can of course find where I use those folders in the code and change it, but it's probably easier to just create them :)

# Steps

## Getting The Data

The first thing we have to do is actually get our data. Navigate to data.itelescope.net, and log in. Next find the images for your observation, then download and unzip them. 

From there, we should open up one of the images in DS9 (See OnQ if unsure whet that program is) and check what we have for a peak value on the target.

For this example, I used image 6, and had a peak of 37k.

Next we need to rename the images, which would be a bit tedious to do manually.

## Renaming Script
With this script I get all of my descriptive file names renamed into '###.fit' in a folder called 'input', which is what we'll need for our next script. There was/is a seperate notebook for this, but it is very simple so I added a few cells to do the same thing here.

When you do this, you'll need to replace the `downloaded_images_path` with the path that will be relevant for you. This will be different, because you'll have taken images on a different day at the very least, so just change it. There are 4 commented lines showin how I navigated from my root project folder to where the images are, and we then move them by 'renaming' them.

In [2]:
import os

In [3]:
#print(os.listdir())
#print(os.listdir('T21'))
#print(os.listdir('T21/TrES-2'))
#print(os.listdir('T21/TrES-2/20230807'))
downloaded_images_path = 'T21/TrES-2/20230807/'

In [4]:
for filename in os.listdir(downloaded_images_path):
    #The last seven digits of our images will be ###.fit, so that's what we keep
    os.rename(downloaded_images_path+filename,"input/"+filename[-7:])

## AstrometryNotebook (Req: client.py)

For the next major step, you'll need to open up a suplementary notebook: "AstrometryNotebook". You'll also want to make sure you have the "client.py" module installed somewhere. It should be included in the QAOP modules folder, but the subnotebook will have a new python kernal and not know to look there, so you'll have to copy the same path you used to this new notebook.

This notebook is quite well detailed! Basically we're adding all of our images to nova.astrometry.net to get wcs data.
The basic summary of the process is that the notebook will take every image in the 'input' folder, and then upload them to the site. It will then loop through the list it created of all the files it uploaded, and download the new versions. These just appear in the same directory the script is in, so the last thing it does is move them into the folder 'output'.

## StarIDNotebook
The next step of our analysis is to identify the stars, and give them names that we will use in our analysis.

We don't particularly care or need to know their actual names at this point in our analysis, so I generally name them all like 'sA,sB,...'. 

The one difference is that we need to identify which one is the target, so I will often add another identifier. The point of all of this is to have a list of RA and DEC for stars that we will use in our future analysis.

All of this will be done using the "StarIDNotebook" at the moment, although a QAOP module is in the works to do this functionality. For now though, open up the suplementary notebook, and follow it's steps to create the `nameloc.csv` file.

*It is pretty badly documented at the moment, but I am going to prioritize finishing the documentation of all the things and steps in the process before I really go back and explain that whole mess... It is, of all the suplementary notebooks, the last reamining corpse of my original mega notebook, which means it has a lot of confusing extra things and hurts my brain to look at...*

## Radial Profiling Notebook

After we have given names to each of our stars, the final step in preparing for photometry is to know how big of an aperture to use. We'll do this using a relative scale based on the Full Width Half Max of the stars in our images, which we will find with this notebook.

After going through this notebook, you should have created an `apertures.csv` file

*I totally forgot about detailing this notebook as I was writing and explaining in the master, and I had thought I was done and ready to upload some to github, but then realized I forgot it. This was a mental blow, and I shall leave properly explaining for the first update and continue uploading to github*

## Photometry

The next step is perhaps my favourite, and, as of the current moment, the most 'behind the scenes'-y. 

The process does exist in the `codeDemo` notebook, but it is done using the `QAOP_photometry` package, and we can run that from here.

Right now there is no tutorial/guide, so to understand you'll have to look through the module (which is extremely well documented). I do however plan on eventually making a notebook to introduce and guide the process, provided I have the time to do so.

If you use the notebook, you can skip the following code cells.

In [ ]:
import QAOP.QAOP_photometry as QPhot
import numpy as np
#import os #We should have os from above.
from astropy.table import Table

In [ ]:
photinstance = QPhot.photInstance(apertureFilePath='apertures.csv')

In [ ]:
#This version makes your times be out of order, though that can be sorted later. Uncomment and run these lines instead
#.  of those in the cell below if you want to do it this way.

#Now, we need to run this photometry. We can do that by calling the runForFile method with a file path
#for fn in os.listdir('output'):
#    filepath = 'output/'+fn
#    photinstance.runForFile(filepath) #def runForFile(self,filepath):

In [ ]:
#Now, we need to run this photometry. We can do that by calling the runForFile method with a file path,
#we can get that file path with a helpful little function in the utils module
from QAOP.QAOP_utils import getFilepath
#This version counts up numbers; 
highest_image_number = 195

for imgnumber in range(1,highest_image_number+1):
    #if imgnumber == {badnumber}: continue #Skip any numbers that we need to (ex, file corrupted etc)
    #if imgnumber == {badnumber2}: continue
    filepath = getFilepath(imgnumber)
    photinstance.runForFile(filepath) #function is: runForFile(self,filepath)

In [ ]:
#After we have run for all of our files
photinstance.exportMasterAsSimple()

## Plotting

Now, with the conclusion of that step, you should have done all the steps you need in order to plot your light curve.
You'll need to do the differential photometry step still as well, but I've been doing that in my plotting notebook.

For an example, I have included my plotting notebook, but you'll need different plots each time, and so I didn't comment or prepare any scripts for that part.


As a basic summary of what you'll need:

You read in the data by using `photData = Table.read('photometry/simple.ecsv)`. From there, you can access the data that we found. There's a column, `time` which has datetime strings that are pretty funky and we can't use to plot, so for your x-axis you'll need to get just the numeric times out of there. There's a function you can import from the utils package to convert them, if you just pass it the data table you have: `times = genTimesFromTable(photData)`

Next, you can get the fluxes by simply referencing the data with the name of the star you want, for example, if I wanted the y-data to plot flux vs time for my target, I would get it from `photData['target'].data`. Because the data is stored in an astropy table, just using `photData['target']` would give us a subtable with the header type info, which we can probably still plot, but just to be safe I like to extract an array of the values with the `.data` property.

With all that said, I could then plot the data in the way that I am most familiar with.

To do the differential part, we can use either (or both) of two ways; relative flux, and magnitudes.

If you would like to compute the magnitude difference between two stars, there is a function in the utils package `diffMag(flux1,flux2)` that will give you that.  For the relative flux, simply divide the flux of the star you want (probably the target) by the flux of your other comparison star. if you have numpy arrays, it will handle the element-wise stuff automatically and you can just have `relative_flux = flux1/flux2` in your code.